## Import libraries
Developing in Jupyter Lab, so using JupyterDash

In [1]:
import pandas as pd
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
import plotly.express as px
import censoring

## Configure for JupyterDash

In [3]:
JupyterDash.infer_jupyter_proxy_config()

## Create Summary Data Frame

In the future, more intricate callbacks will be used on a single data frame. A summary data frame is suitable for my purposes

In [4]:
df = pd.read_csv(censoring.wordle_output)
# limit to the puzzle in which we last have words (396)
df2 = df[ (df['PuzzleNum'] < 447) & (df['PuzzleNum'] != 421) & (df['Difficulty'] != 'Undefined')].copy()
df3 = df2.groupby(['Name','Fails']).size().reset_index()
df3 = df3.rename(columns={ df3.columns[2]: "Counts" })
player_list = df3['Name'].unique().tolist()

## App w/Single Callback

In [6]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/dZVMbK.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div(
    children=[
        html.H1(children='Wordle Scoreboard & Analytics'),
        html.Div(children='Fails By player'),
        html.Div([
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in player_list],
                value='Choose a Player'
            ),
            html.Div([
                dcc.Graph(id='bars-fail-distribution')], 
                style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'})
        ])
    ]
)

@app.callback(
    dash.dependencies.Output('bars-fail-distribution', 'figure'),
    dash.dependencies.Input('crossfilter-xaxis-column', 'value')
)
def update_figure(selected_player):
    filtered_df = df3[df3.Name == selected_player]

    fig = px.bar(
        filtered_df, 
        x="Fails", 
        y="Counts", 
        color="Name", 
        barmode="group"
    )

    fig.update_layout(transition_duration=500)

    return fig


## Run App

In [7]:
app.run_server(debug="True", mode="jupyterlab")